# Main imports and code

In [ ]:
# check which gpu we're using
!nvidia-smi

Sun Jan 30 07:57:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

     |████████████████████████████████| 248 kB 13.7 MB/s 
     |████████████████████████████████| 1.7 MB 66.6 MB/s 
     |████████████████████████████████| 9.7 MB 58.1 MB/s 
     |████████████████████████████████| 3.5 MB 62.0 MB/s 
     |████████████████████████████████| 1.2 MB 59.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 6.8 MB 54.8 MB/s 
     |████████████████████████████████| 312 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 895 kB 66.8 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 180 kB 62.9 MB/s 
     |████████████████████████████████| 143 kB 74.5 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |████████████████████████████████| 133 kB 77.4 MB/s 
     |████████████████

     |████████████████████████████████| 124 kB 13.6 MB/s 


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval

ModuleNotFoundError: ignored

In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [ ]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


# Fetch Don't Patronize Me! data manager module

In [ ]:
from urllib import request
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
dpm.load_test()
len(dpm.test_set)

3828

In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')

# Load test data

In [ ]:
# load test data
tedf = pd.read_csv("task4_test.tsv", sep='\t', header = None)
tedf_text = tedf[tedf.columns[4]]
tedf_text

0       In the meantime , conservatives are working to...
1       In most poor households with no education chil...
2       The real question is not whether immigration i...
3       In total , the country 's immigrant population...
4       Members of the church , which is part of Ken C...
                              ...                        
3827    In a letter dated Thursday to European Commiss...
3828    They discovered that poor families with health...
3829    She married at 19 , to Milan ( Emil ) Badovina...
3830    The United Kingdom is n't going to devolve int...
3831    This moral battle informed the recent defectio...
Name: 4, Length: 3832, dtype: object

# Load Task1 data

In [ ]:
# This method loads the subtask 1 data
dpm.load_task1()
# which we can then access as a dataframe
dpm.train_task1_df.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [ ]:
trdf1 = dpm.train_task1_df
Counter(trdf1.label)

Counter({0: 9476, 1: 993})

In [ ]:
# oversample positive instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,pcldf,pcldf,pcldf,pcldf,pcldf,pcldf,pcldf,pcldf,trdf1[trdf1.label==0][:npos*9]])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
training_set1

,par_id,art_id,keyword,country,text,label,orig_label
32,33,@@8301323,disabled,pk,arshad said that besides learning many new asp...,1,2
33,34,@@24150149,disabled,ng,fast food employee who fed disabled man become...,1,3
41,42,@@4591694,hopeless,jm,vanessa had feelings of hopelessness in her fi...,1,3
76,77,@@22454828,homeless,nz,"in september , major nottle set off on foot fr...",1,3
82,83,@@4672144,homeless,pk,the demographics of pakistan and india are ver...,1,3
...,...,...,...,...,...,...,...
9871,9872,@@1850907,vulnerable,tz,this caf ? will serve as a platform for buildi...,0,0
9872,9873,@@1886875,refugee,tz,"the report examines the laws , policies and pr...",0,0
9873,9874,@@1884514,poor-families,ng,"according to him , the state and federal gover...",0,1
9874,9875,@@4564081,homeless,jm,"columns <h> poor , pregnant and homeless",0,1


In [ ]:
Counter(training_set1.label)

Counter({0: 8937, 1: 8937})

# Task1 Model

In [ ]:
task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      max_seq_length=500,
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set1[['text', 'label']])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/17874 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2235 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(2235, 0.4233561985178015)

In [ ]:
# run predictions
preds_task1, _ = task1_model.predict(tedf_text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3832 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [ ]:
Counter(preds_task1)

Counter({0: 3274, 1: 558})

In [ ]:
labels2file([[k] for k in preds_task1], 'task1.txt')

# Load Task2 data

In [ ]:
# Same for subtask 2
dpm.load_task2(return_one_hot=True)
# which we can then access as a dataframe
dpm.train_task2_df

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
988,873,@@20374243,citing the fact that these kids who died at go...,poor-families,sg,"[1, 0, 0, 0, 1, 0, 0]"
989,10070,@@15573661,fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[1, 0, 0, 0, 0, 0, 0]"
990,6484,@@2559173,"touched much by their plight , commanding offi...",homeless,lk,"[1, 0, 0, 0, 0, 1, 0]"
991,6249,@@1947926,she reiterated her ministry 's commitment to p...,women,gh,"[1, 0, 0, 0, 0, 0, 0]"


In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  label = trids.label[idx]
  # select row from original dataset to retrieve the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  rows2.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
trdf2 = pd.DataFrame(rows2)
trdf2.label = trdf2.label.apply(literal_eval)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  label = teids.label[idx]
  #print(parid)
  # select row from original dataset to access the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  rows2.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
tedf2 = pd.DataFrame(rows2)
tedf2.label = tedf2.label.apply(literal_eval)

In [ ]:
# concat
train_df2 = tedf2.append(trdf2)
train_df2

,par_id,text,label
0,4046,we also know that they can benefit by receivin...,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,pope francis washed and kissed the feet of mus...,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,many refugees do n't want to be resettled anyw...,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,"""budding chefs , like """" fred """" , """" winston ...","[1, 0, 0, 1, 1, 1, 0]"
4,4089,"""in a 90-degree view of his constituency , one...","[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...
8370,8380,rescue teams search for survivors on the rubbl...,"[0, 0, 0, 0, 0, 0, 0]"
8371,8381,the launch of ' happy birthday ' took place la...,"[0, 0, 0, 0, 0, 0, 0]"
8372,8382,"the unrest has left at least 20,000 people dea...","[0, 0, 0, 0, 0, 0, 0]"
8373,8383,you have to see it from my perspective . i may...,"[0, 0, 0, 0, 0, 0, 0]"


In [ ]:
all_negs = trdf2[trdf2.label.apply(lambda x:sum(x) == 0)]
all_pos = trdf2[trdf2.label.apply(lambda x:sum(x) > 0)]

training_set2 = pd.concat([all_pos,all_pos,all_pos,all_pos,all_pos,all_pos,all_pos,all_pos,all_pos,all_negs[:round(len(all_pos)*9)]])

In [ ]:
training_set2

,par_id,text,label
0,4341,"the scheme saw an estimated 150,000 children f...","[1, 0, 0, 1, 0, 0, 0]"
1,4136,durban 's homeless communities reconciliation ...,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,the next immediate problem that cropped up was...,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,far more important than the implications for t...,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,to strengthen child-sensitive social protectio...,"[1, 0, 0, 1, 1, 1, 0]"
...,...,...,...
7935,7899,"this year , samaritan 's purse hopes to reduce...","[0, 0, 0, 0, 0, 0, 0]"
7936,7900,"its deputy minister , datuk ab aziz kaprawi sa...","[0, 0, 0, 0, 0, 0, 0]"
7937,7901,there were similarly joyous scenes in the city...,"[0, 0, 0, 0, 0, 0, 0]"
7938,7902,the former head of a care home for the mentall...,"[0, 0, 0, 0, 0, 0, 0]"


# Task2 Model

In [ ]:
task2_model_args = MultiLabelClassificationArgs(num_train_epochs=1,
                                                no_save=True, 
                                                no_cache=True, 
                                                max_seq_length=500,
                                                overwrite_output_dir=True
                                                )
task2_model = MultiLabelClassificationModel("roberta", 
                                            'roberta-base', 
                                            num_labels=7,
                                            args = task2_model_args, 
                                            use_cuda=cuda_available)
# train model
task2_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task2, _ = task2_model.predict(tedf_text.tolist())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'c

  0%|          | 0/14292 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1787 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3832 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [ ]:
labels2file(preds_task2, 'task2.txt')

# Prepare submission

In [ ]:
!cat task1.txt | head -n 10

0
1
0
0
0
1
0
0
0
0


In [ ]:
!cat task2.txt | head -n 10

0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0


In [ ]:
!zip submission.zip task1.txt task2.txt

  adding: task1.txt (deflated 93%)
  adding: task2.txt (deflated 98%)
